In [1]:
import nltk
import numpy as np
import random
import string # to process standard python strings
import os, json
import pandas as pd

In [5]:
# Get question and nested answer 
data = []
with open('data/climatology-1.json') as json_file:
    data = json.load(json_file)
count = 1

def traverseAnswerObjects(answersObjects):
    print("answer: " + answersObjects["text"])
    print("\n")
    if(len(answersObjects["answers"])):
        for answerObject in answersObjects["answers"]:
            traverseAnswerObjects(answerObject)

for questionObject in data:
    print("Question: "+ questionObject["question"])
    print("\n")
    traverseAnswerObjects(questionObject)

    count += 1
    if(count == 2):
        break

Question: Are Ice Ages Affected by the Freezing Out of CO2 in Antarctica?


answer: The freezing point of carbon dioxide is -78.5 o o C. Occasionally this low is exceeded in Antarctica, and small quantities of CO2 are frozen out, but are returned soon after to the atmosphere. Is it possible that during an ice age more substantial quantities of CO2 are frozen out, and could it be enough to affect the climate and worsen the ice age? Could this phenomenon also occur in Siberia?


answer: Re Occasionally this low is exceeded in Antarctica, and small quantities of CO2 are frozen out, but are returned soon after to the atmosphere. That is incorrect, and it is based on the value of -78.5deg C. The correct value is about -140deg C, and that's for current amount of carbon dioxide in the atmosphere. Carbon dioxide levels are considerably less than current during glaciations ("ice ages"), which reduces the temperature even further.


answer: As a point of reference, people have been continuously 